In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 4, Finished, Available, Finished)

In [3]:
# Pull the clean data from the Lakehouse bridge
df = spark.read.table("cleaned_retail_data")

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 5, Finished, Available, Finished)

In [4]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/Retail 2010-11.csv"
df = pd.read_csv("/lakehouse/default/Files/Retail 2010-11.csv")

df.head()
df.tail()
df.shape
df.info()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 6, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541910 non-null  object 
 1   StockCode    541910 non-null  object 
 2   Description  540456 non-null  object 
 3   Quantity     541910 non-null  int64  
 4   InvoiceDate  541910 non-null  object 
 5   Price        541910 non-null  float64
 6   Customer ID  406830 non-null  float64
 7   Country      541910 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
#Standardize column names after converting them into string type!!

df.columns = (
    df.columns.astype(str)
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace(r"[^\w_]", "", regex=True)
)
df.info()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 7, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   invoice      541910 non-null  object 
 1   stockcode    541910 non-null  object 
 2   description  540456 non-null  object 
 3   quantity     541910 non-null  int64  
 4   invoicedate  541910 non-null  object 
 5   price        541910 non-null  float64
 6   customer_id  406830 non-null  float64
 7   country      541910 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
df.duplicated().sum()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 8, Finished, Available, Finished)

5268

In [7]:
df = df.drop_duplicates()
df.duplicated().sum()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 9, Finished, Available, Finished)

0

In [8]:
#Handling missing values
df.isna().sum()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 10, Finished, Available, Finished)

invoice             0
stockcode           0
description      1454
quantity            0
invoicedate         0
price               0
customer_id    135037
country             0
dtype: int64

In [9]:
df_clean  = df.dropna()
df_clean.isna().sum()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 11, Finished, Available, Finished)

invoice        0
stockcode      0
description    0
quantity       0
invoicedate    0
price          0
customer_id    0
country        0
dtype: int64

In [10]:
df_clean.head()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 12, Finished, Available, Finished)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [11]:
#Sanity Check
df_clean[df_clean['quantity'] <= 0].shape
df_clean[df_clean['price'] <= 0].shape


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 13, Finished, Available, Finished)

(40, 8)

In [12]:
df_clean[['quantity', 'price']].describe(percentiles=[.01, .05, .95, .99])
df_clean.groupby('invoice').size().sort_values(ascending=False).head(10)
df_clean.duplicated().sum()


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 14, Finished, Available, Finished)

0

In [13]:
df_clean['revenue'] = df_clean['quantity'] * df_clean['price']
df_clean[df_clean['revenue'] < 0].shape

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 15, Finished, Available, Finished)

/tmp/ipykernel_5813/1831155725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['revenue'] = df_clean['quantity'] * df_clean['price']


(8872, 9)

In [14]:
df_clean[df_clean['revenue'] > df_clean['revenue'].quantile(0.999)]


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 16, Finished, Available, Finished)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,revenue
870,536477,21137,BLACK RECORD COVER FRAME,480,01-12-2010 12:27,3.39,16210.0,United Kingdom,1627.2
2364,536584,84029E,RED WOOLLY HOTTIE WHITE HEART.,384,01-12-2010 16:22,2.95,13777.0,United Kingdom,1132.8
4505,536785,22423,REGENCY CAKESTAND 3 TIER,144,02-12-2010 15:22,10.95,15061.0,United Kingdom,1576.8
4850,536809,84950,ASSORTED COLOUR T-LIGHT HOLDER,1824,02-12-2010 16:48,0.55,15299.0,United Kingdom,1003.2
4946,536830,21915,RED HARMONICA IN BOX,1400,02-12-2010 17:38,1.06,16754.0,United Kingdom,1484.0
...,...,...,...,...,...,...,...,...,...
540061,581457,23543,WALL ART KEEP CALM,698,08-12-2011 18:43,4.15,18102.0,United Kingdom,2896.7
540062,581457,82484,WOOD BLACK BOARD ANT WHITE FINISH,204,08-12-2011 18:43,4.80,18102.0,United Kingdom,979.2
540063,581457,23553,LANDMARK FRAME CAMDEN TOWN,120,08-12-2011 18:43,10.95,18102.0,United Kingdom,1314.0
540070,581458,22197,POPCORN HOLDER,1500,08-12-2011 18:45,0.72,17949.0,United Kingdom,1080.0


In [15]:
df_clean.groupby('stockcode')['price'].nunique().sort_values(ascending=False).head(10)


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 17, Finished, Available, Finished)

stockcode
M         184
D          75
POST       71
DOT        16
CRUK       16
21166      12
82484      10
21175      10
85099B      9
85152       9
Name: price, dtype: int64

In [16]:
df_clean['country'].value_counts()


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 18, Finished, Available, Finished)

country
United Kingdom          356728
Germany                   9480
France                    8476
EIRE                      7475
Spain                     2528
Netherlands               2371
Belgium                   2069
Switzerland               1877
Portugal                  1471
Australia                 1258
Norway                    1086
Italy                      803
Channel Islands            757
Finland                    695
Cyprus                     611
Sweden                     461
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
USA                        291
Israel                     247
Unspecified                241
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon                     45


In [17]:
# Fixing Data Types

df_clean['invoice']= df_clean['invoice'].astype('string')
df_clean['stockcode'] = df_clean['stockcode'].astype('string')
df_clean['description'] = df_clean['description'].astype('string')
df_clean['quantity']= df_clean['quantity'].astype('Int64')
df_clean["invoicedate"] = pd.to_datetime(df_clean["invoicedate"], errors="coerce")
df_clean['price']= pd.to_numeric(df_clean['price'], errors='coerce')
df_clean['customer_id']= df_clean['customer_id'].astype('int64')
df_clean['country'] = df_clean['country'].astype('string')


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 19, Finished, Available, Finished)

/tmp/ipykernel_5813/561799328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['invoice']= df_clean['invoice'].astype('string')
/tmp/ipykernel_5813/561799328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['stockcode'] = df_clean['stockcode'].astype('string')
/tmp/ipykernel_5813/561799328.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [18]:
df_clean.dtypes

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 20, Finished, Available, Finished)

invoice        string[python]
stockcode      string[python]
description    string[python]
quantity                Int64
invoicedate    datetime64[ns]
price                 float64
customer_id             int64
country        string[python]
revenue               float64
dtype: object

In [19]:
df_clean.isna().sum()


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 21, Finished, Available, Finished)

invoice             0
stockcode           0
description         0
quantity            0
invoicedate    231112
price               0
customer_id         0
country             0
revenue             0
dtype: int64

In [20]:
df_clean[df_clean['stockcode'].isin(['POST','M','D'])][['stockcode','description']].drop_duplicates()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 22, Finished, Available, Finished)

,stockcode,description
45,POST,POSTAGE
141,D,Discount
2239,M,Manual


In [21]:
#Flag missing dates

df_clean['is_missing_date'] = df_clean['invoicedate'].isna()

df_clean[df_clean['is_missing_date']].head()


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 23, Finished, Available, Finished)

/tmp/ipykernel_5813/1555053556.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['is_missing_date'] = df_clean['invoicedate'].isna()


,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,revenue,is_missing_date
26732,538521,21754,HOME BUILDING BLOCK WORD,3,NaT,5.95,14180,United Kingdom,17.85,True
26733,538521,21755,LOVE BUILDING BLOCK WORD,3,NaT,5.95,14180,United Kingdom,17.85,True
26734,538521,22072,RED RETROSPOT TEA CUP AND SAUCER,8,NaT,3.75,14180,United Kingdom,30.00,True
26735,538521,22846,BREAD BIN DINER STYLE RED,1,NaT,16.95,14180,United Kingdom,16.95,True
26736,538521,22849,BREAD BIN DINER STYLE MINT,1,NaT,16.95,14180,United Kingdom,16.95,True


In [22]:
df_clean[df_clean['is_missing_date']]['quantity'].describe()



StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 24, Finished, Available, Finished)

count      231112.0
mean      12.246093
std      224.251063
min        -74215.0
25%             2.0
50%             5.0
75%            12.0
max         74215.0
Name: quantity, dtype: Float64

In [23]:
df_clean[df_clean['is_missing_date']]['revenue'].describe()

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 25, Finished, Available, Finished)

count    231112.000000
mean         20.593284
std         240.978277
min      -77183.600000
25%           4.250000
50%          11.800000
75%          19.800000
max       77183.600000
Name: revenue, dtype: float64

## Flags creation

In [24]:
df_clean['is_missing_date'] = df_clean['invoicedate'].isna()
df_clean['is_cancelled'] = df_clean['invoice'].str.startswith('C')
df_clean['is_return'] = df_clean['quantity'] < 0
df_clean['is_zero_price'] = df_clean['price'] <= 0
df_clean['is_non_product'] = df_clean['stockcode'].isin(['POST', 'M', 'D'])

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 26, Finished, Available, Finished)

/tmp/ipykernel_5813/3920048265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['is_missing_date'] = df_clean['invoicedate'].isna()
/tmp/ipykernel_5813/3920048265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['is_cancelled'] = df_clean['invoice'].str.startswith('C')
/tmp/ipykernel_5813/3920048265.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [25]:
df_clean[
    ['is_missing_date','is_cancelled','is_return','is_zero_price','is_non_product']
].mean() * 100


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 27, Finished, Available, Finished)

is_missing_date    57.547092
is_cancelled        2.209136
is_return           2.209136
is_zero_price        0.00996
is_non_product      0.431768
dtype: Float64

In [26]:
df_clean['is_time_usable'] = ~df_clean['is_missing_date']


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 28, Finished, Available, Finished)

/tmp/ipykernel_5813/3621987983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['is_time_usable'] = ~df_clean['is_missing_date']


In [27]:
df_clean.groupby('is_missing_date')['revenue'].sum()


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 29, Finished, Available, Finished)

is_missing_date
False    3519182.480
True     4759354.944
Name: revenue, dtype: float64

~**57%** of total revenue comes from <u>transactions</u> without invoice dates

~**43%** comes from <u>transactions</u> with valid dates

A significant portion of revenue is tied to transactions **without** reliable timestamps, which makes time-series analysis incomplete unless explicitly scoped.

###### #**Crucial**: Defining views instead of deleting rows!!

In [28]:
df_gross = df_clean.copy()
df_gross.head()


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 30, Finished, Available, Finished)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,revenue,is_missing_date,is_cancelled,is_return,is_zero_price,is_non_product,is_time_usable
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850,United Kingdom,15.30,False,False,False,False,False,True
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850,United Kingdom,20.34,False,False,False,False,False,True
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850,United Kingdom,22.00,False,False,False,False,False,True
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850,United Kingdom,20.34,False,False,False,False,False,True
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850,United Kingdom,20.34,False,False,False,False,False,True


## Net sales (what business actually sold)

In [29]:
df_net = df_clean[
    ~df_clean['is_cancelled'] &
    ~df_clean['is_return'] &
    ~df_clean['is_zero_price']
]

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 31, Finished, Available, Finished)

## Time-usable sales (for trends)

In [30]:
df_time = df_net[df_net['is_time_usable']]


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 32, Finished, Available, Finished)

## Export as **Parquet**

In [31]:
df_clean.to_parquet(
    "/lakehouse/default/Files/silver/retail_transactions.parquet",
    engine="pyarrow",
    index=False
)


StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 33, Finished, Available, Finished)

In [32]:
import os
os.listdir("/lakehouse/default/Files/silver")

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 34, Finished, Available, Finished)

['retail_transactions.parquet']

In [33]:
# 1. Convert finished Pandas dataframe back to Spark
final_spark_df = spark.createDataFrame(df_clean)

# 2. Save it as the final table that Power BI is looking for
final_spark_df.write.format("delta").mode("overwrite").saveAsTable("final_retail_transactions")

StatementMeta(, a7424682-0bdc-4ac3-944a-1e1ae963df7a, 35, Finished, Available, Finished)